In [10]:
import numpy as np
import pandas as pd
import pandas_datareader as wb
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from scipy.stats import norm
import seaborn as sns

sns.set()

These are the standard modules which i generally use while doing any analysis which over time I have got from trial and error

## Assignment 1 - Getting Data

In [2]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile

Date_1 = ['2022-03-01','2022-03-02','2022-03-03']
data = pd.DataFrame()

for r in Date_1:
    ######connects to the link to download the zipfile
    link = urlopen("https://data.binance.vision/data/spot/daily/trades/BNBUSDT/BNBUSDT-trades-" + r + ".zip")
    files = ZipFile(BytesIO(link.read()))

    #####Opens the csv and read into pandas dataframe - data
    data_temp = pd.read_csv(files.open('BNBUSDT-trades-' + r + '.csv'),names = ['Trade_ID','price','qty','quoteqty','time',
                                                                     'isBuyerMaker','isBestMatch'])
    data = pd.concat([data,data_temp])

In [3]:
data

,Trade_ID,price,qty,quoteqty,time,isBuyerMaker,isBestMatch
0,522804734,395.7,0.038,15.0366,1646092800113,False,True
1,522804735,395.7,0.029,11.4753,1646092800118,False,True
2,522804736,395.7,0.027,10.6839,1646092800193,False,True
3,522804737,395.7,0.037,14.6409,1646092800199,False,True
4,522804738,395.7,0.038,15.0366,1646092800428,False,True
...,...,...,...,...,...,...,...
320900,524236011,402.6,0.027,10.8702,1646351997768,False,True
320901,524236012,402.5,1.369,551.0225,1646351998669,True,True
320902,524236013,402.6,1.150,462.9900,1646351999052,False,True
320903,524236014,402.6,0.125,50.3250,1646351999432,False,True


## Assignment 2

In [4]:
data_1 = data.drop_duplicates(subset=['time'],keep='first') ###########It drops double duplicate times or double quotes

In [5]:
data_1

,Trade_ID,price,qty,quoteqty,time,isBuyerMaker,isBestMatch
0,522804734,395.7,0.038,15.0366,1646092800113,False,True
1,522804735,395.7,0.029,11.4753,1646092800118,False,True
2,522804736,395.7,0.027,10.6839,1646092800193,False,True
3,522804737,395.7,0.037,14.6409,1646092800199,False,True
4,522804738,395.7,0.038,15.0366,1646092800428,False,True
...,...,...,...,...,...,...,...
320900,524236011,402.6,0.027,10.8702,1646351997768,False,True
320901,524236012,402.5,1.369,551.0225,1646351998669,True,True
320902,524236013,402.6,1.150,462.9900,1646351999052,False,True
320903,524236014,402.6,0.125,50.3250,1646351999432,False,True


In [6]:
Margin = 0.0003 #########This is the assumptions 3 bps margin on bid and offer
size_USD = 1000 #######Bid-offer size
max_position = 5000 ######## Max position allowed


Order_ID = 0

latency = 100

In [7]:

#########the function takes input as a series from trade data and gives the list of parameters as bid#####


def quote(data,quote_type,Margin=.0003,size_USD=1000,latency=100,i=[0]):
    
    if quote_type == 0:                            #######this is an indicator for bid =0 or offer ==1

        Price = np.round(data.price-Margin,4)               #########Gets the bid price
        size = np.round(size_USD/Price,3)                   ######The bid size in BTC
        Quote_Qty = np.round(size*Price,4)    ##### The bid quote qty in USD upto 4 decimals
        Order_time  = data.time+100           ##### Order when it reaches the exchange with latency of 100 ms
        Order_type = 'B'                             ##### Distinguishes between types of order 'B'-bid,'O'-Offer
        Trade   = 0
        i[0]    += 1                                ########Counts the number of times the function order is called
        Trade_time = 0 
        #########Currently 0 when the trade will be executed it will show trade_time + 100ms latency for when
        ######## the market maker would come to know###### 
        Trade_exit_price = 0             ##############Trade_exit_price same for bid-offer, different for hedging
        
    
        
        return pd.DataFrame([i[0],Order_type,Price,size,Quote_Qty,Order_time,Trade,Trade_time,Trade_exit_price],
                            index=['ID','Order_type','Price','size_BTC','Quote_Qty_USD','Time','Trade_Status','Trade_known_time','Trade_exit_price']).T
    
    elif quote_type == 1:
        
        Price = np.round(data.price+Margin,4)               #########Gets the offer price
        size = np.round(size_USD/Price,3)                   ######The offer size in BTC
        Quote_Qty = np.round(size*Price,4)    ##### The offer quote qty in USD upto 4 decimals
        Order_time  = data.time+100           ##### Order when it reaches the exchange with latency of 100 ms
        Order_type = 'O'                             ##### Distinguishes between types of order 'B'-bid,'O'-Offer
        Trade   = 0
        i[0]    += 1                                ########Counts the number of times the function order is called
        
        Trade_time = 0 
        Trade_exit_price = 0  
        
        return pd.DataFrame([i[0],Order_type,Price,size,Quote_Qty,Order_time,Trade,Trade_time,Trade_exit_price],
                            index=['ID','Order_type','Price','size_BTC','Quote_Qty_USD','Time','Trade_Status','Trade_known_time','Trade_exit_price']).T
   
    else:
        
        Price =  0                                         #########Market order
        size = size_USD                          ########The qty in BTC###########direct
        Quote_Qty = np.round(size*data.price,4)    ##### The offer quote qty in USD upto 4 decimals
        Order_time  = data.time+100           ##### Order when it reaches the exchange with latency of 100 ms
        Order_type = quote_type                             ##### Distinguishes between types of order 'B'-bid,'O'-Offer
        Trade   = 0
        i[0]    += 1                                ########Counts the number of times the function order is called
        
        Trade_time = data.time+100 
        Trade_exit_price = 0  
        
        return pd.DataFrame([i[0],Order_type,Price,size,Quote_Qty,Order_time,Trade,Trade_time,Trade_exit_price],
                            index=['ID','Order_type','Price','size_BTC','Quote_Qty_USD','Time','Trade_Status','Trade_known_time','Trade_exit_price']).T
   
        

In [8]:
##################This is if hedge is 0##############
############This funcction checks if the trade happened by price crossing or not#############
#############it takes as input data, our trade data checks for  just two recent prices#####
########### Open order book##################
#####it returns a list with trade match == 1 or else 0 (If no trade occurs), 
#### Trade time with 100 ms latency as market maker comes to know 100ms late
### ID's of the order book
### Trade_exit_price

def trade_signal(data,Open_order,hedge=0):
    
     if hedge==0:
        
        pricep = data.iloc[len(data)-2:len(data)].price.to_numpy()    
        trade = []
        time = []
        ID = []
        temp = Open_order.Price.to_numpy()
        check = np.diff(np.sign(pricep-temp[:,None])).flatten().nonzero()[0]
         ###### This checks for a crossover & returns an  array with index of numbers
            
        time  = [(data.iloc[-1].time+100)] * len(check)
             ########## 100 second latency as market maker will come to know of trade only 100 ms later

        trade = [1]* len(check)
        ID    =    Open_order.ID.iloc[check].tolist()
       
        return ID,trade,time    
   
     else:     ###########It takes data as a single series
        temp    = Open_order.Price.tolist()
        pricep  = [data.price] * len(temp)
        time    = []
        ID      = []
        trade   = []
        time    = [data.time] * len(temp)  
            ####IN case of hedging orders the latency of from trade execution to market maker is omitted 
            #######as here, the market maker knows he has sent a market order to the exchange though he may be not
            #####sure of price of execution which is fine for us as we only use the knowledge of hedge order sent
            #####for our calcuations to calculate the hedge
            
            
        trade    = [1] * len(temp)               ##########Trade status
        ID       =  Open_order.ID.tolist()  ###########ID
            
        return ID,trade,time,pricep    

In [9]:
                    ########This initialises the trade till the first trade reaches the exchange after 213 ms
Order_Book= pd.DataFrame()    
for r in range(4):
    Order_Book = pd.concat([Order_Book,quote(data_1.iloc[r],0)])
    Order_Book = pd.concat([Order_Book,quote(data_1.iloc[r],1)])

#########This initialises
Open_order = Order_Book

In [77]:
Order_Book

,ID,Order_type,Price,size_BTC,Quote_Qty_USD,Time,Trade_Status,Trade_known_time,Trade_exit_price
0,1,B,395.6997,2.527,999.9331,1646092800213,0,0,0
0,2,O,395.7003,2.527,999.9347,1646092800213,0,0,0
0,3,B,395.6997,2.527,999.9331,1646092800218,0,0,0
0,4,O,395.7003,2.527,999.9347,1646092800218,0,0,0
0,5,B,395.6997,2.527,999.9331,1646092800293,0,0,0
0,6,O,395.7003,2.527,999.9347,1646092800293,0,0,0
0,7,B,395.6997,2.527,999.9331,1646092800299,0,0,0
0,8,O,395.7003,2.527,999.9347,1646092800299,0,0,0


Conditions:
1) Quote to be used only when sending orders to the exchange (As it appends the ID)
2) For trade function only use recent two prices,, always use :x+1 to get the price

In [16]:
for x in range(4,124744):
            ####It checks for open orders

    Open_order = Order_Book[Order_Book['Time'].values <= data_1.time.iloc[x]]
    
    if len(Open_order)== 0:
        Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],0))])
        Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],1))])
    else:    
    
        
    
    
        ##############completes hedging trade if any#################
        Hedge_order = Open_order[Open_order['Price'].values == 0]
        if len(Hedge_order) != 0:
            ID,trade_status,time,trade_exit_price = trade_signal(data_1.iloc[x],Hedge_order,hedge=1)
            Order_Book.loc[Order_Book.ID.isin(ID),'Trade_Status'] = trade_status
            Order_Book.loc[Order_Book.ID.isin(ID),'Trade_known_time'] = time
            Order_Book.loc[Order_Book.ID.isin(ID),'Trade_exit_price'] = trade_exit_price
        #################hedging as required##############

    
        Open_order = Open_order[Open_order['Trade_known_time'].values == 0] 
        Open_order = Open_order[Open_order['Price'].values != 0 ]
        #############Eliminates cancelled & Hedged trades trades which as a result us in having only open trades or
        ############ trades for which prices have crossed in the exchange but market maker has no idea latency,However, the latter such trades will not harm us as if we send to cancel the exchange they may be already executed 
       ##########For knowing the max open position we have already adjusted
    
        ###########Initially the loop will check for max_Position
    
        
        
        ##############max position#####
        Trade_Book = Order_Book[((Order_Book.Trade_Status.values==1) & (Order_Book.Trade_known_time.values <= data_1.time.iloc[x]))|
                               ((Order_Book.Price.values==0) & (Order_Book.Time.values-100 <= data_1.time.iloc[x])& (Order_Book.Trade_Status.values==0))]
       #########Our trade book or inventory book only considers
          ######## 1) Executed orders as per trader 2) Those hedges (Market orders) sent to the exchange but the market maker will come to know about execution only after 100ms
        ###########BUt he knows he has sent an open there
        ##########eliminates those trades which he will come to know only 100ms later
        Trade_Book_Bid = Trade_Book[Trade_Book.Order_type.values == 'B'] ###Executed trades bid_book
        Trade_Book_Offer = Trade_Book[Trade_Book.Order_type.values == 'O'] ### Executed trades offer_Book
        Max_qty = Trade_Book_Bid.size_BTC.values.sum()-Trade_Book_Offer.size_BTC.values.sum() #### Qty of total bitcoins
        Max_Position = Max_qty *data_1.price.values[x]
        
        ########### Case 1#############################
        
        if Max_Position >= max_position: #############5000USD
            
              ######Stop quoting on the bid side#####
            Open_order_bid = Open_order[Open_order.Order_type.values =='B']
            Open_order_bid = Open_order_bid[Open_order_bid.Trade_Status.values==0]
            Order_Book.loc[Order_Book.ID.isin(Open_order_bid.ID),'Trade_Status'] = -1
            
            Order_Book.loc[Order_Book.ID.isin(Open_order_bid.ID),'Trade_known_time'] = data_1.time.iloc[x] + 100
            ####Taking latency when we issue the orders to the exchange 
            ##### Cancelled orders are -1
            
            ##########Hedging###################### 
            ##For passive hedging another quote pair is required USDTBTC hence may be I dont understand the point clearly
            ## Hence will use market order
            Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],'O',size_USD=Max_qty))])
            
            ##########Continues the offer quote
            Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],1))])
        
        
        ############  Case 2 ###########

        elif Max_Position <= -max_position: ##############5000 USD
        
               ######Stop quoting on the offer side#####
            Open_order_offer = Open_order[Open_order.Order_type.values =='O']
            Open_order_offer = Open_order_offer[Open_order_offer.Trade_Status.values==0]
            Order_Book.loc[Order_Book.ID.isin(Open_order_offer.ID),'Trade_Status'] = -1
            Order_Book.loc[Order_Book.ID.isin(Open_order_offer.ID),'Trade_known_time'] = data_1.time.iloc[x] + 100
            ####Taking latency when we issue the orders to the exchange 
            ##### Cancelled orders are -1        


            ##########For passive hedging another quote pair is required USDTBTC hence may be I dont understand the point clearly
            ########## Hence will use market order
            Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],'B',size_USD=-Max_qty))])
    
    
            ##########Continues the bid quote
            Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],0))])
        
         
            ########### Case 3 ################
    
        else:    
               #############BId- Offer#########Quote
            Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],0))])
            Order_Book = pd.concat([Order_Book,(quote(data_1.iloc[x],1))])
        
        

            ######### checks if trade price crosses our bid-offer quotes###############
        ID,trade_status,trade_known_time = trade_signal(data_1.iloc[:x+1],Open_order)
        Order_Book.loc[Order_Book.ID.isin(ID),'Trade_Status'] = trade_status
        Order_Book.loc[Order_Book.ID.isin(ID),'Trade_known_time'] = trade_known_time 
        
        ###########this also cares of the fact if we have cancelled orders and they reach the exchange 100ms as in above 
       ########### step but before that trade happens in  the market will take precedence (###Though we may over/under hedge)
    
        Order_Book =  Order_Book[Order_Book.Trade_Status.values != -1]
        #########IT drops the rows with -1 to increase speed and memory

KeyboardInterrupt: 

In [20]:
x

124744

In [21]:
Order_Book

,ID,Order_type,Price,size_BTC,Quote_Qty_USD,Time,Trade_Status,Trade_known_time,Trade_exit_price
0,1,B,395.6997,2.527,999.9331,1646092800213,1,1646092800538,0
0,2,O,395.7003,2.527,999.9347,1646092800213,1,1646092804992,0
0,3,B,395.6997,2.527,999.9331,1646092800218,1,1646092800538,0
0,4,O,395.7003,2.527,999.9347,1646092800218,1,1646092804992,0
0,5,B,395.6997,2.527,999.9331,1646092800293,1,1646092800538,0
...,...,...,...,...,...,...,...,...,...
0,249484,O,415.3003,2.408,1000.0431,1646122170288,0,0,0
0,249485,B,415.2997,2.408,1000.0417,1646122170478,0,0,0
0,249486,O,415.3003,2.408,1000.0431,1646122170478,0,0,0
0,249487,B,415.2997,2.408,1000.0417,1646122170636,0,0,0


##### Important
In case of latency what really effects us is the problem of those trades which are executed by the exchange and 
we think they are open only in case of Max position determination

#### 
The code became extremely slow despite repeated efforts using different packages....Considering the time constraints it Assignment three was was done with data till rows 124744/828176 which is till this time (2022,3,1,13,39,30,842000)

# Assignment 3 - Its just half day

In [27]:
Trades = Order_Book[Order_Book['Trade_Status']==1]

In [97]:
Trades

,ID,Order_type,Price,size_BTC,Quote_Qty_USD,Time,Trade_Status,Trade_known_time,Trade_exit_price
0,1,B,395.6997,2.527,999.9331,1646092800213,1,1646092800538,0
0,2,O,395.7003,2.527,999.9347,1646092800213,1,1646092804992,0
0,3,B,395.6997,2.527,999.9331,1646092800218,1,1646092800538,0
0,4,O,395.7003,2.527,999.9347,1646092800218,1,1646092804992,0
0,5,B,395.6997,2.527,999.9331,1646092800293,1,1646092800538,0
...,...,...,...,...,...,...,...,...,...
0,249464,O,415.3003,2.408,1000.0431,1646122169289,1,1646122169550,0
0,249465,B,415.3997,2.407,999.8671,1646122169310,1,1646122169550,0
0,249468,O,415.3003,2.408,1000.0431,1646122169332,1,1646122169550,0
0,249469,B,415.3997,2.407,999.8671,1646122169550,1,1646122169672,0


In [99]:
######We will split the trades into half######### to consider two Days or two time periods
Trades_5 = Trades.iloc[:79669]
Trades_6 = Trades.iloc[79668:]

Trades_7 = [Trades_5,Trades_6]

In [104]:
Dollar_volume = []
P_L = []
no_of_trades = []

for x in range(len(Trades_7)):
    Trades = Trades_7[x]
    Trades_1 = Trades[Trades['Price']!=0]
    Trades_Bid,Trades_Offer,Trades_Hedge = Trades_1[Trades_1['Order_type']=='B'], Trades_1[Trades_1['Order_type']=='O'],Trades[Trades['Price']==0]
    ########Trades_Bid,Trades_Offer,Trades_Hedge 
    
    Dollar_volume.append(np.round(Trades_Bid.Quote_Qty_USD.sum() + Trades_Offer.Quote_Qty_USD.sum()+ (Trades_Hedge.Trade_exit_price * Trades_Hedge.size_BTC).sum()))
    ###########Dollar_volume form Trades_Bid,Trades_Offer,Trades_Hedged(Buy&Sold)
    
    Trades_Hedge_Offer,Trades_Hedge_Bid = Trades_Hedge[Trades_Hedge['Order_type']=='O'],Trades_Hedge[Trades_Hedge['Order_type']=='B']
     
    P_L.append(Trades_Offer.Quote_Qty_USD.sum()+ (Trades_Hedge_Offer.size_BTC * Trades_Hedge_Offer.Trade_exit_price).sum()-Trades_Bid.Quote_Qty_USD.sum()-(Trades_Hedge_Bid.size_BTC * Trades_Hedge_Bid.Trade_exit_price).sum())
    
    no_of_trades.append(len(Trades))

In [140]:
Results = pd.DataFrame([Dollar_volume,np.round(P_L),no_of_trades],index=['Dollar_volume','Gross_P_L_$','no_of_trades'],columns=['day_1','day_2'],dtype='object').T

In [142]:
Results['Gross_P_L']=(Results['Gross_P_L_$']/Results['Dollar_volume'])*100000

In [143]:
Results

,Dollar_volume,Gross_P_L_$,no_of_trades,Gross_P_L
day_1,123049923.0,12620.0,79669,10.256
day_2,120938105.0,-54410.0,79670,-44.989956
